In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

### Task 1: Logging function

In [3]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    with open('code_log.txt', 'a') as f:
        f.write(f'{datetime.now()}: {message}\n')

In [56]:
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = ["Name", "MC_USD_Billion"]
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = './exchange_rate.csv'
output_path = './Largest_banks_data.csv'

### Task 2 : Extraction of data

In [46]:
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''

    #Extract the web page as text
    html_page = requests.get(url).text

    #Parse the html page    
    data = BeautifulSoup(html_page, 'html.parser')

    #Create an empty pandas DataFrame named df with columns as the table_attribs
    df = pd.DataFrame(columns=table_attribs)

    #Extract all 'tbody' attributes of the HTML object and then extract all the rows of the index 2 table using the 'tr' attribute.
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')

    #Check the contents of each row, having attribute ‘td’, for the following conditions.
    #a. The row should not be empty.
    #b. The first column should contain a hyperlink.
    #c. The third column should not be '—'.

    rows_to_add = []

    for row in rows:
                 
    #     # Extract all 'td' elements in the current row
         cells = row.find_all('td')
   
         if len(cells) > 2 and cells[1].find('a'):
                    
          # Extract data for each attribute
          bank = cells[1].text.strip()
          mc_usd_billions = float(cells[2].text.strip())

          # Store valid entries in a dictionary
          entry = {
                 table_attribs[0]: bank,
                 table_attribs[1]: mc_usd_billions if mc_usd_billions else None
                 }
          # Add the entry to the list of rows to add
          rows_to_add.append(entry)
    #Concatenate all rows to the DataFrame
    df = pd.concat([df, pd.DataFrame(rows_to_add)], ignore_index=True)

    return df

In [47]:
df=extract(url, table_attribs)
print(df)

                                 Bank_name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                        HSBC Holdings PLC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81


/tmp/ipykernel_21287/2804197234.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(rows_to_add)], ignore_index=True)


### Task 3 : Transformation of data

In [50]:
exchange_rate_df=pd.read_csv('exchange_rate.csv')
exchange_rate = exchange_rate_df.set_index('Currency').to_dict()['Rate']

{'EUR': 0.93, 'GBP': 0.8, 'INR': 82.95}


In [53]:
df['MC_GBP_Billion'] = [np.round(x*exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
df['MC_EUR_Billion'] = [np.round(x*exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
df['MC_INR_Billion'] = [np.round(x*exchange_rate['INR'],2) for x in df['MC_USD_Billion']]
print(df)  

                                 Bank_name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

In [55]:
df['MC_EUR_Billion'][4]

np.float64(146.86)

### Task 4: Loading to CSV

In [57]:
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path, index=False)

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    query_result  = pd.read_sql_query(query_statement, sql_connection)
    print(query_statement)
    print(query_result)

In [58]:
load_to_csv(df, output_path)
log_progress('Data saved to CSV file')

#Initiate SQLite3 connection
sql_connection = sqlite3.connect(db_name)
log_progress('	SQL Connection initiated')

#Call load_to_db() function
load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database as a table, Executing queries')

#Call run_query() function
query_statement = 'SELECT * FROM Largest_banks'
run_query(query_statement, sql_connection)
query_statement = 'SELECT AVG(MC_GBP_Billion) FROM Largest_banks'
run_query(query_statement, sql_connection)
query_statement = 'SELECT Name from Largest_banks LIMIT 5'
run_query(query_statement, sql_connection)
log_progress('Process Complete')

#Close SQLite3 connection
log_progress('Server Connection closed')

SELECT * FROM Largest_banks
                                 Bank_name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2    

DatabaseError: Execution failed on sql 'SELECT Name from Largest_banks LIMIT 5': no such column: Name